In [3]:
from utils import getStringBetweenTwoWords, extract_and_stringify_object , printJsonToFile,printSoupToHtml
from Product import ProductDetailDTO
#Dependencies
from bs4 import BeautifulSoup
import requests
import json

async def scrape_aliexpress_full(url: str) -> dict:
    # with open("aliexpressX.html", 'r', encoding='utf-8') as file:
    #     soup = BeautifulSoup(file, 'html.parser')
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    printSoupToHtml(soup,"aliexpressR.html")
    from requests_html import AsyncHTMLSession
    asession = AsyncHTMLSession(
        mock_browser=True
    )
    response = await asession.get(url)
    soup = response.content
    printSoupToHtml(soup,"aliexpressAR.html")
    #try another library
    # import httpx
    # response = httpx.get(url)
    # printSoupToHtml(response,"aliexpressAR.html")

    script_tag = soup.find('script')
    if script_tag:
        script_content = script_tag.string
        script_target_object= getStringBetweenTwoWords(script_content, 'window.runParams =', 'window.runParams.is23').replace(';', '')
        script_target_object = script_target_object.replace('data', '"data"')
        script_target_object = json.loads(script_target_object)
        price = script_target_object['data']['metaDataComponent']['ogTitle'].split('|')[0].strip()
        title = script_target_object['data']['metaDataComponent']['title'].split('|')[0].strip()
        ImageList = script_target_object['data']['imageComponent']['imagePathList']
        description = script_target_object['data']['productPropComponent']['propGroups']
        attr_values = []
    
        for group in description:
            for prop in group.get("props", []):
                attr_key = prop.get("attrName", "").strip()
                attr_value = prop.get("attrValue", "").strip()
                if attr_value:
                    attr_values.append(attr_key+":"+attr_value)
        
        concatenated_description = ', '.join(attr_values)

    item_data = ProductDetailDTO(name_Global=title, price=price,images=ImageList,productlink1=url,description_Global=concatenated_description)
    return item_data

url = 'https://www.aliexpress.com/item/1005006860843367.html?spm=a2g0o.productlist.main.1.4cbdDLvvDLvvt5&algo_pvid=7246bb87-519c-45f0-9096-37d5a5e6b81c&utparam-url=scene%3Asearch%7Cquery_from%3A&gatewayAdapt=glo2ara'
x=await scrape_aliexpress_full(url)
print("name_global:",x.name_Global)
print("isAvailable:",x.is_available)    
print("price:",x.price)
print("rating:",x.rating)
print("description:",x.description_Global)
print("images:",x.images)




TypeError: argument should be integer or bytes-like object, not 'str'

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from utils import getStringBetweenTwoWords, extract_and_stringify_object , printJsonToFile,printSoupToHtml
from Product import ProductDetailDTO
#Dependencies
from bs4 import BeautifulSoup
import requests
import json

async def scrape_aliexpress_full(url: str) -> dict:
    options = Options()
    options.add_argument("--headless")  # Run Chrome in headless mode
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # just go to url and get the page source
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    # wait for the page to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'body'))
    )
    script_target_object= getStringBetweenTwoWords(driver.page_source, 'type="application/ld+json">', '</script>').replace(';', '')
    script_target_object = json.loads(script_target_object)

    #soup = BeautifulSoup(driver.page_source, 'html.parser')
    name_Global = script_target_object[0]['name']
    is_available = script_target_object[0]['offers']['availability'] == 'https://schema.org/InStock'
    price = script_target_object[0]['offers']['price']
    description_Global = script_target_object[0]['description']
    images = script_target_object[0]['image']
    return {
        'name_Global': name_Global,
        'is_available': is_available,
        'price': price,
        'description_Global': description_Global,
        'images': images
    }
    
    printSoupToHtml(script_target_object,"aliexpressRx.html")
url = 'https://www.aliexpress.com/item/1005006860843367.html?spm=a2g0o.productlist.main.1.4cbdDLvvDLvvt5&algo_pvid=7246bb87-519c-45f0-9096-37d5a5e6b81c&utparam-url=scene%3Asearch%7Cquery_from%3A&gatewayAdapt=glo2ara'
x = await scrape_aliexpress_full(url)
print(x)
# print("name_global:", x.name_Global)
# print("isAvailable:", x.is_available)
# print("price:", x.price)
# print("rating:", x.rating)
# print("description:", x.description_Global)
# print("images:", x.images)

{'name_Global': 'Apple-iPhone 15 Pro ، A3104 ، APPLE ، A17 Pro ، Nano SIM ، IP68 ، إصدار CN ، علامة تجارية جديدة ونشطة ، أصلية ، منتجات أصلية - AliExpress 509', 'is_available': True, 'price': 50511.63, 'description_Global': 'شراء Apple-iPhone 15 Pro ، A3104 ، APPLE ، A17 Pro ، Nano SIM ، IP68 ، إصدار CN ، علامة تجارية جديدة ونشطة ، أصلية ، منتجات أصلية في Aliexpress مقابل . ابحث عن المزيد من منتجات 509 و 5090301 و . استمتع بشحن مجاني في جميع أنحاء العالم! ✓ بيع محدود الوقت ✓ إرجاع سهل.', 'images': ['https://ae01.alicdn.com/kf/S43878c1648584e11ab92142ecc68f0b9h/Apple-iPhone-15-Pro-A3104-APPLE-A17-Pro-Nano-SIM-IP68-CN.jpg', 'https://ae01.alicdn.com/kf/S7fd4c9dc78df49559248bece1647335c7/Apple-iPhone-15-Pro-A3104-APPLE-A17-Pro-Nano-SIM-IP68-CN.png', 'https://ae01.alicdn.com/kf/S6a96d337ef514436ad758a123ebe190dq/Apple-iPhone-15-Pro-A3104-APPLE-A17-Pro-Nano-SIM-IP68-CN.png']}
